In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import arviz as az

import stan_jupyter as stan
import pandas as pd

In [50]:
def waic(fit):

    log_lik = [n.mean() for n in fit['log_lik']]

    lppd = np.log(np.exp(log_lik).mean(axis=0)).sum()

    p_waic = np.var(log_lik, axis=0).sum()

    waic = -2*lppd + 2*p_waic

    return round(waic, 3)

## Question 1

### FROM https://github.com/pymc-devs/resources/blob/master/Rethinking_2/Chp_06.ipynb ###

In [15]:
def inv_logit(x):
    return np.exp(x) / (1 + np.exp(x))


def sim_happiness(N_years=100, seed=1234):
    np.random.seed(seed)

    popn = pd.DataFrame(np.zeros((20 * 65, 3)), columns=["age", "happiness", "married"])
    popn.loc[:, "age"] = np.repeat(np.arange(65), 20)
    popn.loc[:, "happiness"] = np.repeat(np.linspace(-2, 2, 20), 65)
    popn.loc[:, "married"] = np.array(popn.loc[:, "married"].values, dtype="bool")

    for i in range(N_years):
        # age population
        popn.loc[:, "age"] += 1
        # replace old folk with new folk
        ind = popn.age == 65
        popn.loc[ind, "age"] = 0
        popn.loc[ind, "married"] = False
        popn.loc[ind, "happiness"] = np.linspace(-2, 2, 20)

        # do the work
        elligible = (popn.married == 0) & (popn.age >= 18)
        marry = np.random.binomial(1, inv_logit(popn.loc[elligible, "happiness"] - 4)) == 1
        popn.loc[elligible, "married"] = marry

    popn.sort_values("age", inplace=True, ignore_index=True)

    return popn



In [16]:
popn = sim_happiness()

df = popn.copy()
df["married"] = df["married"].astype(
    int
)  # this is necessary before using az.summary, which doesn't work with boolean columns.
az.summary(df.to_dict(orient="list"), kind="stats", round_to=2)

mean     sd  hdi_3%  hdi_97%
age        32.00  18.77     0.0    61.00
happiness  -0.00   1.21    -2.0     1.79
married     0.28   0.45     0.0     1.00

__Model 6.9__

In [18]:
df.married +=1

In [55]:
df = df[df.age>17]
df["age_adj"] = (df.age - 18) / (65-18)

In [85]:
model_data = {"age":df.age_adj.tolist(),
              "happiness":df.happiness.tolist(),
              "mid":df.married.tolist(),
              "N":len(df),
              "MIDS":df.married.nunique()}

with open("models/w4_1.stan") as f:
    model_code = f.read()

In [86]:
%%capture
posterior = stan.build(model_code, model_data)

In [87]:
%%capture
fit = posterior.sample(num_chains=4, num_samples=1000)

In [88]:
az.summary(fit)[0:4]

mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
alpha[0] -0.200  0.065  -0.323   -0.079      0.002    0.001    1547.0   
alpha[1]  1.216  0.090   1.049    1.384      0.002    0.002    1563.0   
beta     -0.718  0.120  -0.950   -0.502      0.003    0.002    1444.0   
sigma     1.021  0.023   0.979    1.065      0.000    0.000    2472.0   

          ess_tail  r_hat  
alpha[0]    2337.0    1.0  
alpha[1]    2261.0    1.0  
beta        1833.0    1.0  
sigma       1958.0    1.0

In [89]:
m6_9 = az.from_pystan(fit, log_likelihood="log_lik")

__Model 6.10__

In [90]:
model_data = {"age":df.age_adj.tolist(),
              "happiness":df.happiness.tolist(),
              "N":len(df)}

with open("models/w4_1b.stan") as f:
    model_code = f.read()

In [91]:
%%capture
posterior = stan.build(model_code, model_data)

In [92]:
%%capture
fit = posterior.sample(num_chains=4, num_samples=1000)

In [93]:
az.summary(fit)[0:3]

mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
alpha -0.001  0.077  -0.139    0.150      0.002    0.001    1699.0    1942.0   
beta   0.003  0.135  -0.238    0.270      0.003    0.002    1734.0    2101.0   
sigma  1.216  0.028   1.161    1.267      0.001    0.000    2212.0    1905.0   

       r_hat  
alpha    1.0  
beta     1.0  
sigma    1.0

In [94]:
m6_10 = az.from_pystan(fit, log_likelihood="log_lik")

In [98]:
compare_dict = {"with marriage ID": m6_9, "pooled": m6_10}
az.compare(compare_dict)

/Users/jikaasik/miniconda/lib/python3.9/site-packages/arviz/stats/stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(


rank          loo     p_loo       d_loo    weight  \
with marriage ID     0 -1355.379693  3.547407    0.000000  0.979123   
pooled               1 -1518.741901  2.381088  163.362207  0.020877   

                         se       dse  warning loo_scale  
with marriage ID  18.658002   0.00000    False       log  
pooled            13.670496  16.52657    False       log

In [99]:
az.compare(compare_dict, ic="waic")

/Users/jikaasik/miniconda/lib/python3.9/site-packages/arviz/stats/stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(


rank         waic    p_waic      d_waic    weight  \
with marriage ID     0 -1355.379619  3.547333    0.000000  0.979122   
pooled               1 -1518.741791  2.380979  163.362172  0.020878   

                         se        dse  warning waic_scale  
with marriage ID  18.658001   0.000000    False        log  
pooled            13.670492  16.526572    False        log

According to the WAIC and PSIS values, the model that differentiates between married and unmarried observations should make better out-of-sample predictions. 

The model believes that someone at the minimum age in the sample (18) is considerably more likely to be happy if they are married. Both married and unmarried individuals become less happy as they age. 

## Question 2

In [108]:
df = pd.read_csv("../data/foxes.csv", delimiter=";")

df["std_food"] = (df.avgfood - df.avgfood.mean()) / df.avgfood.std()

model_data = {"N":len(df),
              "weight":df.weight.tolist(),
              "food":df.std_food.tolist()}

with open("models/w4_2.stan") as f:
    model_code = f.read()

In [110]:
%%capture
posterior = stan.build(model_code, model_data)

In [111]:
%%capture
fit = posterior.sample(num_chains=4, num_samples=1000)

In [112]:
m4_2 = az.from_pystan(fit, log_likelihood="log_lik")

In [114]:
df["std_groupsize"] = (df.groupsize - df.groupsize.mean()) / df.groupsize.std()

model_data = {"N":len(df),
              "weight":df.weight.tolist(),
              "food":df.std_food.tolist(),
              "group_size":df.std_groupsize.tolist()}

with open("models/w4_2b.stan") as f:
    model_code = f.read()

In [116]:
%%capture
posterior = stan.build(model_code, model_data)

In [117]:
%%capture
fit = posterior.sample(num_chains=4, num_samples=1000)

In [118]:
m4_2b = az.from_pystan(fit, log_likelihood="log_lik")

In [121]:
compare_dict = {"Pooled": m4_2, "Stratified by group size": m4_2b}
az.compare(compare_dict)

/Users/jikaasik/miniconda/lib/python3.9/site-packages/arviz/stats/stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(


rank         loo     p_loo     d_loo    weight  \
Stratified by group size     0 -181.302510  3.688649  0.000000  0.874428   
Pooled                       1 -186.473301  2.513736  5.170791  0.125572   

                                se       dse  warning loo_scale  
Stratified by group size  7.866228  0.000000    False       log  
Pooled                    6.674532  3.647028    False       log

In [122]:
az.compare(compare_dict, ic="waic")

/Users/jikaasik/miniconda/lib/python3.9/site-packages/arviz/stats/stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(
/Users/jikaasik/miniconda/lib/python3.9/site-packages/arviz/stats/stats.py:1405: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


rank        waic    p_waic    d_waic    weight  \
Stratified by group size     0 -181.288785  3.674924  0.000000  0.875427   
Pooled                       1 -186.470497  2.510932  5.181712  0.124573   

                                se       dse  warning waic_scale  
Stratified by group size  7.862314  0.000000     True        log  
Pooled                    6.674125  3.644015    False        log

In [124]:
az.summary(m4_2b)

mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
alpha         4.524  0.104   4.323    4.716      0.002    0.001    3250.0   
beta          0.681  0.234   0.252    1.117      0.005    0.003    2501.0   
gamma        -0.796  0.231  -1.213   -0.358      0.005    0.003    2545.0   
sigma         1.138  0.076   0.993    1.274      0.001    0.001    2895.0   
mu[0]         4.427  0.230   4.008    4.880      0.004    0.003    3760.0   
...             ...    ...     ...      ...        ...      ...       ...   
log_lik[111] -1.110  0.069  -1.243   -0.987      0.001    0.001    2824.0   
log_lik[112] -1.142  0.069  -1.272   -1.016      0.001    0.001    3089.0   
log_lik[113] -1.379  0.186  -1.712   -1.050      0.003    0.003    3035.0   
log_lik[114] -1.699  0.260  -2.181   -1.244      0.005    0.003    3046.0   
log_lik[115] -1.085  0.083  -1.243   -0.936      0.002    0.001    2709.0   

              ess_tail  r_hat  
alpha           2549.0    1.0  
beta            2313.0    1.0  
gamma           2476.0    1.0  
sigma           2242.0    1.0  
mu[0]           3449.0    1.0  
...                ...    ...  
log_lik[111]    2230.0    1.0  
log_lik[112]    2557.0    1.0  
log_lik[113]    3244.0    1.0  
log_lik[114]    3205.0    1.0  
log_lik[115]    2859.0    1.0  

[236 rows x 9 columns]

In this case, the model that stratifies by group size will be a better out-of-sample predictor according to both information criteria. Since both predictors are standardized, the intercept `alpha` represents expected weight when both predictors are at their mean values. The beta coefficient means a standard deviation change in the average quantity of food available corresponds to an expected weight of +.68 units. The gamma coefficient means that a standard deviation change in group size corresponds to an expected weight of -.8 units. The standard deviations for both parameters are quite large, which suggests the model is not particularly certain about the magnitude of the effects, though it is quite certain about the direction.

## Question 3